DeFi BI-ETL Pipeline - Exploratory Data Analysis
This file can be converted to a Jupyter notebook using:
jupytext --to notebook exploration.py

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [ ]:
# Add src to path for imports
sys.path.insert(0, str(Path(__file__).parent.parent / "src"))

In [ ]:
from utils.io import get_processed_data_path, get_tableau_path
from utils.time import utc_now

In [ ]:
# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
print("DeFi BI-ETL Pipeline - Exploratory Data Analysis")
print("=" * 60)
print(f"Analysis started at: {utc_now()}")
print()

=============================================================================
1. LOAD AND EXPLORE DATA
=============================================================================

In [ ]:
print("1. LOADING AND EXPLORING DATA")
print("-" * 40)

In [ ]:
# Check what data files are available
processed_dir = Path("data/processed")
tableau_dir = Path("dashboards/tableau")

In [ ]:
print("Available processed data files:")
if processed_dir.exists():
    for csv_file in processed_dir.glob("*.csv"):
        print(f"  - {csv_file.name}")
else:
    print("  No processed data directory found")

In [ ]:
print("\nAvailable Tableau export files:")
if tableau_dir.exists():
    for csv_file in tableau_dir.glob("*.csv"):
        print(f"  - {csv_file.name}")
else:
    print("  No Tableau export directory found")

In [ ]:
print()

=============================================================================
2. MARKETS DATA ANALYSIS
=============================================================================

In [ ]:
print("2. MARKETS DATA ANALYSIS")
print("-" * 40)

In [ ]:
try:
    markets_file = get_processed_data_path("cg_markets_latest.csv")
    if Path(markets_file).exists():
        markets_df = pd.read_csv(markets_file)
        print(f"Markets data loaded: {len(markets_df)} records, {len(markets_df.columns)} columns")
        
        # Basic statistics
        print(f"\nMarket Cap Statistics:")
        print(f"  Total Market Cap: ${markets_df['market_cap'].sum() / 1e9:.2f}B")
        print(f"  Average Market Cap: ${markets_df['market_cap'].mean() / 1e9:.2f}B")
        print(f"  Median Market Cap: ${markets_df['market_cap'].median() / 1e9:.2f}B")
        
        # Top 10 by market cap
        print(f"\nTop 10 Cryptocurrencies by Market Cap:")
        top_10 = markets_df.nlargest(10, 'market_cap')[['symbol', 'name', 'market_cap', 'pct_24h']]
        for _, row in top_10.iterrows():
            market_cap_b = row['market_cap'] / 1e9
            pct_change = row['pct_24h']
            print(f"  {row['symbol']:>8} ({row['name'][:20]:<20}) ${market_cap_b:>8.2f}B  {pct_change:>6.2f}%")
        
        # Price change distribution
        print(f"\n24h Price Change Distribution:")
        pct_24h = markets_df['pct_24h'].dropna()
        print(f"  Mean: {pct_24h.mean():.2f}%")
        print(f"  Median: {pct_24h.median():.2f}%")
        print(f"  Std: {pct_24h.std():.2f}%")
        print(f"  Min: {pct_24h.min():.2f}%")
        print(f"  Max: {pct_24h.max():.2f}%")
        
        # Volume analysis
        print(f"\nVolume Analysis:")
        total_volume = markets_df['total_volume'].sum()
        print(f"  Total 24h Volume: ${total_volume / 1e9:.2f}B")
        print(f"  Volume/Market Cap Ratio: {(total_volume / markets_df['market_cap'].sum()):.2%}")
        
    else:
        print("Markets data file not found")
        markets_df = None

In [ ]:
except Exception as e:
    print(f"Error loading markets data: {e}")
    markets_df = None

In [ ]:
print()

=============================================================================
3. CATEGORIES DATA ANALYSIS
=============================================================================

In [ ]:
print("3. CATEGORIES DATA ANALYSIS")
print("-" * 40)

In [ ]:
try:
    categories_file = get_processed_data_path("cg_categories_snapshot.csv")
    if Path(categories_file).exists():
        categories_df = pd.read_csv(categories_file)
        print(f"Categories data loaded: {len(categories_df)} records, {len(categories_df.columns)} columns")
        
        # Market cap by category
        print(f"\nMarket Cap by Category:")
        category_mcap = categories_df.groupby('name')['market_cap'].sum().sort_values(ascending=False)
        for category, mcap in category_mcap.head(10).items():
            mcap_b = mcap / 1e9
            print(f"  {category[:30]:<30} ${mcap_b:>8.2f}B")
        
        # Category distribution
        total_mcap = categories_df['market_cap'].sum()
        print(f"\nCategory Market Share:")
        for category, mcap in category_mcap.head(10).items():
            share = (mcap / total_mcap) * 100
            print(f"  {category[:30]:<30} {share:>6.2f}%")
        
    else:
        print("Categories data file not found")
        categories_df = None

In [ ]:
except Exception as e:
    print(f"Error loading categories data: {e}")
    categories_df = None

In [ ]:
print()

=============================================================================
4. TVL DATA ANALYSIS
=============================================================================

In [ ]:
print("4. TVL DATA ANALYSIS")
print("-" * 40)

In [ ]:
try:
    tvl_file = get_processed_data_path("llama_tvl_protocols.csv")
    if Path(tvl_file).exists():
        tvl_df = pd.read_csv(tvl_file)
        print(f"TVL data loaded: {len(tvl_df)} records, {len(tvl_df.columns)} columns")
        
        # Top protocols by TVL
        print(f"\nTop 10 Protocols by TVL:")
        top_tvl = tvl_df.nlargest(10, 'tvl_usd')[['protocol_name', 'tvl_usd', 'change_1d']]
        for _, row in top_tvl.iterrows():
            tvl_b = row['tvl_usd'] / 1e9
            change_1d = row['change_1d']
            print(f"  {row['protocol_name'][:25]:<25} ${tvl_b:>8.2f}B  {change_1d:>6.2f}%")
        
        # TVL statistics
        print(f"\nTVL Statistics:")
        print(f"  Total TVL: ${tvl_df['tvl_usd'].sum() / 1e9:.2f}B")
        print(f"  Average TVL: ${tvl_df['tvl_usd'].mean() / 1e9:.2f}B")
        print(f"  Median TVL: ${tvl_df['tvl_usd'].median() / 1e9:.2f}B")
        
        # 24h change analysis
        change_1d = tvl_df['change_1d'].dropna()
        print(f"\n24h TVL Change Distribution:")
        print(f"  Mean: {change_1d.mean():.2f}%")
        print(f"  Median: {change_1d.median():.2f}%")
        print(f"  Positive changes: {(change_1d > 0).sum()} protocols")
        print(f"  Negative changes: {(change_1d < 0).sum()} protocols")
        
    else:
        print("TVL data file not found")
        tvl_df = None

In [ ]:
except Exception as e:
    print(f"Error loading TVL data: {e}")
    tvl_df = None

In [ ]:
print()

=============================================================================
5. HISTORICAL TVL ANALYSIS
=============================================================================

In [ ]:
print("5. HISTORICAL TVL ANALYSIS")
print("-" * 40)

In [ ]:
try:
    tvl_hist_file = get_processed_data_path("llama_tvl_protocols_30d.csv")
    if Path(tvl_hist_file).exists():
        tvl_hist_df = pd.read_csv(tvl_hist_file)
        print(f"Historical TVL data loaded: {len(tvl_hist_df)} records, {len(tvl_hist_df.columns)} columns")
        
        # Convert date column
        tvl_hist_df['date'] = pd.to_datetime(tvl_hist_df['date'])
        
        # Date range
        print(f"\nData Date Range:")
        print(f"  Start: {tvl_hist_df['date'].min().strftime('%Y-%m-%d')}")
        print(f"  End: {tvl_hist_df['date'].max().strftime('%Y-%m-%d')}")
        print(f"  Days: {(tvl_hist_df['date'].max() - tvl_hist_df['date'].min()).days}")
        
        # Protocols covered
        protocols = tvl_hist_df['protocol_name'].nunique()
        print(f"\nProtocols covered: {protocols}")
        
        # TVL trend analysis
        print(f"\nTVL Trend Analysis:")
        daily_tvl = tvl_hist_df.groupby('date')['tvl_usd'].sum()
        print(f"  First day total TVL: ${daily_tvl.iloc[0] / 1e9:.2f}B")
        print(f"  Last day total TVL: ${daily_tvl.iloc[-1] / 1e9:.2f}B")
        
        if len(daily_tvl) > 1:
            total_change = ((daily_tvl.iloc[-1] - daily_tvl.iloc[0]) / daily_tvl.iloc[0]) * 100
            print(f"  Total change: {total_change:+.2f}%")
        
    else:
        print("Historical TVL data file not found")
        tvl_hist_df = None

In [ ]:
except Exception as e:
    print(f"Error loading historical TVL data: {e}")
    tvl_hist_df = None

In [ ]:
print()

=============================================================================
6. DATA QUALITY ASSESSMENT
=============================================================================

In [ ]:
print("6. DATA QUALITY ASSESSMENT")
print("-" * 40)

In [ ]:
# Check for missing values
if markets_df is not None:
    print("Markets Data Quality:")
    missing_data = markets_df.isnull().sum()
    if missing_data.sum() > 0:
        print("  Missing values found:")
        for col, missing in missing_data[missing_data > 0].items():
            print(f"    {col}: {missing} ({missing/len(markets_df)*100:.1f}%)")
    else:
        print("  No missing values found")
    
    # Check for duplicates
    duplicates = markets_df.duplicated(subset=['id']).sum()
    print(f"  Duplicate records: {duplicates}")

In [ ]:
if tvl_df is not None:
    print("\nTVL Data Quality:")
    missing_data = tvl_df.isnull().sum()
    if missing_data.sum() > 0:
        print("  Missing values found:")
        for col, missing in missing_data[missing_data > 0].items():
            print(f"    {col}: {missing} ({missing/len(tvl_df)*100:.1f}%)")
    else:
        print("  No missing values found")

In [ ]:
print()

=============================================================================
7. SUMMARY AND RECOMMENDATIONS
=============================================================================

In [ ]:
print("7. SUMMARY AND RECOMMENDATIONS")
print("-" * 40)

In [ ]:
print("Data Pipeline Status:")
if markets_df is not None:
    print("  ✓ Markets data available")
if categories_df is not None:
    print("  ✓ Categories data available")
if tvl_df is not None:
    print("  ✓ TVL data available")
if tvl_hist_df is not None:
    print("  ✓ Historical TVL data available")

In [ ]:
print("\nRecommendations:")
print("  1. Run 'python -m src.cli quickrun' to ensure all data is up to date")
print("  2. Check data freshness - ensure files are from today")
print("  3. Validate data ranges - look for unusual values")
print("  4. Monitor API rate limits and errors")
print("  5. Consider adding data validation rules")

In [ ]:
print(f"\nAnalysis completed at: {utc_now()}")
print("=" * 60)

=============================================================================
8. PLOTTING FUNCTIONS (commented out for CLI usage)
=============================================================================

In [ ]:
"""
# Uncomment these functions if running in Jupyter notebook

def plot_market_cap_distribution(df):
    '''Plot market cap distribution.'''
    plt.figure(figsize=(12, 6))
    
    # Log scale for better visualization
    plt.hist(df['market_cap'] / 1e9, bins=50, alpha=0.7, edgecolor='black')
    plt.xlabel('Market Cap (Billions USD)')
    plt.ylabel('Frequency')
    plt.title('Distribution of Cryptocurrency Market Caps')
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
    plt.show()

def plot_price_changes(df):
    '''Plot 24h price changes.'''
    plt.figure(figsize=(12, 6))
    
    plt.hist(df['pct_24h'].dropna(), bins=50, alpha=0.7, edgecolor='black')
    plt.xlabel('24h Price Change (%)')
    plt.ylabel('Frequency')
    plt.title('Distribution of 24h Price Changes')
    plt.axvline(x=0, color='red', linestyle='--', alpha=0.7)
    plt.grid(True, alpha=0.3)
    plt.show()

def plot_top_markets(df, top_n=20):
    '''Plot top markets by market cap.'''
    plt.figure(figsize=(14, 8))
    
    top_markets = df.nlargest(top_n, 'market_cap')
    
    plt.barh(range(len(top_markets)), top_markets['market_cap'] / 1e9)
    plt.yticks(range(len(top_markets)), top_markets['symbol'])
    plt.xlabel('Market Cap (Billions USD)')
    plt.title(f'Top {top_n} Cryptocurrencies by Market Cap')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_tvl_trends(df):
    '''Plot TVL trends over time.'''
    if 'date' not in df.columns:
        return
    
    plt.figure(figsize=(14, 8))
    
    # Group by date and protocol
    daily_tvl = df.groupby(['date', 'protocol_name'])['tvl_usd'].sum().unstack()
    
    # Plot top 5 protocols
    top_protocols = daily_tvl.sum().nlargest(5).index
    daily_tvl[top_protocols].plot(figsize=(14, 8))
    
    plt.xlabel('Date')
    plt.ylabel('TVL (USD)')
    plt.title('TVL Trends for Top Protocols')
    plt.legend(title='Protocol')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Example usage:
# if markets_df is not None:
#     plot_market_cap_distribution(markets_df)
#     plot_price_changes(markets_df)
#     plot_top_markets(markets_df)

# if tvl_hist_df is not None:
#     plot_tvl_trends(tvl_hist_df)
"""